<a href="https://colab.research.google.com/github/joexu22/llama2-finetune/blob/main/Notebooks/FineTune_LLaMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook finetunes and uploads a finetuned model

credits: https://twitter.com/Dorialexander/status/1681671177696161794

In [ ]:
# git the github gist for llama training code
# not enough of a scientist to fiddle with all the settings/parameters

!rm -rf 4f344196c5509d97a2288837fe6d8d4a
!git clone https://gist.github.com/joexu22/4f344196c5509d97a2288837fe6d8d4a

In [ ]:
# install dependences

!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install bitsandbytes==0.40.2
!pip install transformers==4.30.2
!pip install trl==0.4.7

In [ ]:
# move to script location

%cd 4f344196c5509d97a2288837fe6d8d4a

In [ ]:
# login to HuggingFace

from huggingface_hub import login
login()

In [ ]:
# run script

!python finetune_llama_v2.py --dataset_name "UrbanJoe/LlamaMaster" --max_steps 500 --merge_and_push True --model_name "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
# merge the model together

from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("results/final_merged_checkpoint")

In [ ]:
# test the model

import torch
from transformers import AutoTokenizer

device = "cuda:0"
model = model.to(device)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")

# prepare the prompt
questions = "How do I impress an interviewer?"
text = f"### Human: {questions} ### Assistant: Hey bud,"
inputs = tokenizer(text, return_tensors="pt").to(device)

# delete extra from training
del inputs['token_type_ids']

# Generate output
with torch.no_grad():
  output = model.generate(**inputs, max_new_tokens=1000)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated text:", generated_text)


In [ ]:
# Upload Model to HuggingFace

model.push_to_hub("llama2-true-llama-master-ultimate")